
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [ ]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

In [ ]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/My Drive/CMPT_CV_lab3'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection

### Data Loader

In [ ]:
from glob import glob
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''

def get_train_test_data(set_name):
  data_dirs = '{}/data/'.format(BASE_DIR)
  train_dirs = '{}/data/train/'.format(BASE_DIR)
  test_dirs = '{}/data/test/*'.format(BASE_DIR)

  json_file = os.path.join(data_dirs, "train.json")
  with open(json_file) as f:
    imgs_anns = json.load(f)

  train_data = {}
  test_data = []

  for idx, v in enumerate(imgs_anns):
    record = {}
    filename = os.path.join(train_dirs, v["file_name"])
    obj1 = {
        "bbox": v["bbox"],
        "bbox_mode": BoxMode.XYWH_ABS,
        "segmentation": v["segmentation"],
        "category_id": 0,
        "iscrowd": v["iscrowd"]
    }

    if not filename in train_data:
      height, width = cv2.imread(filename).shape[:2]
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width
      record["annotations"] = []
      train_data[filename] = record
    train_data[filename]["annotations"].append(obj1)

  for filename in glob(test_dirs):
    record = {}
    height, width = cv2.imread(filename).shape[:2]
    record["annotations"] = []
    record["file_name"] = filename
    record["height"] = height
    record["width"] = width
    test_data.append(record)

  train_data = np.array(list(train_data.values()))

  return train_data, test_data

train_dataset, test_dataset = get_train_test_data("train")

def get_detection_data(set_name):
  if set_name == "train":
    return train_dataset.tolist()
  elif set_name == "test":
    return test_dataset

In [ ]:

'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
for d in ["train", "test"]:
  DatasetCatalog.register("plane_" + d, lambda d = d: get_detection_data(d))
  MetadataCatalog.get("plane_" + d).set(thing_classes=["plane"])
plane_metadata = MetadataCatalog.get("plane_train")



In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''

dataset_dicts = get_detection_data("train")
for data in random.sample(dataset_dicts, 3):
    img = cv2.imread(data["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=plane_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(data)
    cv2_imshow(out.get_image()[:, :, ::-1])

### Set Configs

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("plane_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 850
cfg.SOLVER.STEPS = []        
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 



### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''

os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume = False)
trainer.train()


In [ ]:
#for graphs
# %load_ext tensorboard
# %tensorboard --logdir drive/My\Drive/CMPT_CV_lab3/output

### Evaluation and Visualization

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''

dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
evaluator = COCOEvaluator("plane_train", output_dir=cfg.OUTPUT_DIR)
train_loader = build_detection_test_loader(cfg, "plane_train")
print(inference_on_dataset(predictor.model, train_loader, evaluator))


### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''


## Part 2: Semantic Segmentation

### Data Loader

In [ ]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''
train_set = get_detection_data("train")
data_set = []
def get_dataset(data):
  for idx in data:
    filename = idx['file_name']
    image = cv2.imread(filename)
    annotations = idx['annotations']
    temp_data = {}
    temp_data['file_name'] = filename
    temp1_data = {}
    temp1_data['anns'] = []
    for v in annotations:   
      bbox = v['bbox']
      x_1, y_1, x_2, y_2 = bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]
      x_1, y_1, x_2, y_2 = int(x_1), int(y_1), int(x_2), int(y_2)
      
      img = image[y_1:y_2, x_1:x_2]
      img = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)

      msk = detectron2.utils.visualizer.GenericMask(v['segmentation'], idx['height'], idx['width']).mask
      mask = msk[y_1:y_2, x_1:x_2]
      mask = cv2.resize(mask, (128, 128), interpolation = cv2.INTER_AREA)
      records = {'img': img, 'mask': mask}
       
      temp1_data['anns'].append(records)

    temp_data['annotations'] = temp1_data['anns']
    data_set.append(temp_data)
  return data_set

data_set = get_dataset(train_set)

In [ ]:
def get_instance_sample(data, idx, img=None): 
  for i in range(len(data_set)): 
    if data_set[i]['file_name'] == data['file_name']:
      target_value = data_set[i]['annotations'][idx]
      obj_img = target_value['img']  
      obj_mask = target_value['mask']
      obj_img = np.array(obj_img)
      obj_mask = np.array(obj_mask)
      return obj_img, obj_mask

In [ ]:

'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([
          transforms.ToTensor(), # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]
    data = self.data[idx[0]]
 
    image, mask = get_instance_sample(data, idx[1])
    # print(type(image))

    img, mask = self.numpy_to_tensor(image, mask)
    img = img.reshape((3, 128, 128))
    mask = mask.reshape((1, 128, 128))

    return img, mask

def get_plane_dataset(set_name='train', batch_size=2):
    my_data_list = DatasetCatalog.get("plane_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [ ]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            nn.MaxPool2d(2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch)

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # Encoder
      
        self.input_conv = conv(3, 4)
        self.down1 = down(4, 16)
        self.down2 = down(16, 32)
        self.down3 = down(32, 64)
        self.down4 = down(64, 128)
        self.down5 = down(128, 256)
        self.down6 = down(256, 512)
        
        # Decoder
        
        self.up1 = up(512, 256)
        self.up2 = up(256, 128)
        self.up3 = up(128, 64)
        self.up4 = up(64, 32)
        self.up5 = up(32, 16)
        self.up6 = up(16, 4)

        self.output_conv = conv(4, 1, False) # ReLu activation is removed to keep the logits for the loss function
        

    def forward(self, input):
      y = self.input_conv(input)
      y = self.down1(y)
      y = self.down2(y)
      y = self.down3(y)
      y = self.down4(y)
      y = self.down5(y)
      y = self.down6(y)
      y = self.up1(y)
      y = self.up2(y)
      y = self.up3(y)
      y = self.up4(y)
      y = self.up5(y)
      y = self.up6(y)

      output = self.output_conv(y)
      return output
       

### Training

In [ ]:

'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''

# Set the hyperparameters
num_epochs = 50
batch_size = 4
learning_rate = 0.001
weight_decay = 1e-5

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD

# start the training procedure
for epoch in range(num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
    pred = model(img)
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 8
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)

total_iou = 0
temp = 0
for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cpu().detach()
    mask = torch.unsqueeze(mask, 1)
    pred = model(img).cpu().detach()

    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''

    for i in range(img.shape[0]):
      temp = temp+1
      true_img = np.rint(np.array(nn.Sigmoid()(pred[i])[0])) 

      true_mask = np.rint(np.array(nn.Sigmoid()(mask[i])[0]))
      
      intersection = np.sum(np.logical_and(true_mask, true_img))
      union = np.sum(np.logical_or(true_mask, true_img))
      iou = intersection/union

      total_iou = total_iou + iou 

print("\n #images: {}, Mean IoU: {}".format(temp, total_iou/temp))



In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''

# reference from above given function
batch_size = 3 
loader, dataset = get_plane_dataset('train', batch_size)

img, mask = next(iter(loader))
with torch.no_grad():
  img = img.cuda()
  mask = mask.cpu().detach()
  # mask = torch.unsqueeze(mask, 1)
  pred = model(img).cpu().detach()
  for i in range(batch_size):
    pred_i = pred[i]
    mask_i = mask[i]
    img_i = img[i]
    transform_img = transforms.ToPILImage()(img_i.cpu())
    img_array = np.array(transform_img)
    pred_array = np.rint(np.array(nn.Sigmoid()(pred_i)[0]))
    pred_array = pred_array*255 
    mask_array = np.array(mask_i)[0]
    mask_array = mask_array*255
    cv2_imshow(img_array)
    cv2_imshow(pred_array)
    cv2_imshow(mask_array)

## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [ ]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''

def bbox_calc(bbox, check):
  if check == True:
    x_1, y_1, x_2, y_2 = bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]
  else:
    x_1, y_1, x_2, y_2 = bbox[0], bbox[1], bbox[2], bbox[3]

  x_1, y_1, x_2, y_2 = int(x_1), int(y_1), int(x_2), int(y_2) 
  return x_1, y_1, x_2, y_2

def get_prediction_mask(data):
  filename = data['file_name']
  image = cv2.imread(filename)
  annotations = data['annotations']

  gt_mask, pred_mask= np.zeros([data['height'], data['width']]), np.zeros([data['height'], data['width']])

  size = False
  if len(annotations) == 0:
    size = True

  if size:
    calc = predictor(image)['instances']
    calc_size = len(calc)
    for idx in range(calc_size):
      bbox = np.floor(list(calc[idx]._fields['pred_boxes'])[0].cpu().numpy())
      bbox = np.array(bbox, dtype=uint32)
      check = False
      
      x_1, y_1, x_2, y_2 = bbox_calc(bbox, check)

      img = image[y_1:y_2, x_1:x_2]
      img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
      img = torch.tensor(transforms.ToTensor()(img), device = torch.device('cuda'))
      img = model(torch.unsqueeze(img, 0))
      img = img.cpu().detach().numpy()[0]
      sub_x = x_2-x_1
      sub_y = y_2-y_1
      #resizing to original
      new_img = cv2.resize(img[0], (sub_x, sub_y), interpolation = cv2.INTER_AREA)
      new_img = np.rint(np.array(1/(1 + np.exp(-new_img))))
      new_img = new_img*(1+idx)

      mask = pred_mask[y_1:y_2, x_1:x_2]
      mask[mask == 0] = 100
      #for overlapping planes
      olp = np.minimum(mask, new_img)
      olp[olp == 10] = 0
      pred_mask[y_1:y_2, x_1:x_2] = olp
  else:
    for idx, v in enumerate(annotations):
      bbox = v['bbox']
      check = True
      
      x_1, y_1, x_2, y_2 = bbox_calc(bbox, check)

      img = image[y_1:y_2, x_1:x_2]
      img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
      img = torch.tensor(transforms.ToTensor()(img), device = torch.device('cuda'))
      img = model(torch.unsqueeze(img, 0))
      img = img.cpu().detach().numpy()[0]
      sub_x = x_2-x_1
      sub_y = y_2-y_1
      #resizing to original
      new_img = cv2.resize(img[0], (sub_x, sub_y), interpolation = cv2.INTER_AREA)
      new_img = np.rint(np.array(1/(1 + np.exp(-new_img))))
      new_img = new_img*(1+idx)

      mask = pred_mask[y_1:y_2, x_1:x_2]
      mask[mask == 0] = 100
      #for overlapping planes
      olp = np.minimum(mask, new_img)
      olp[olp == 10] = 0
      pred_mask[y_1:y_2, x_1:x_2] = olp

  for idx, v in enumerate(annotations):
    bbox = v['bbox']
    x_1, y_1, x_2, y_2 = bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]
    x_1, y_1, x_2, y_2 = int(x_1), int(y_1), int(x_2), int(y_2)

    segmentations = v['segmentation']
    new_mask = detectron2.utils.visualizer.GenericMask(segmentations, data['height'], data['width']).mask
    mask1 = gt_mask[y_1:y_2, x_1:x_2]
    mask2 = new_mask[y_1:y_2, x_1:x_2]
    olp = np.maximum(mask1, mask2)
    olp = olp*(1+idx)
    gt_mask[y_1:y_2, x_1:x_2] = olp

  gt_mask = torch.tensor(gt_mask, device = torch.device('cuda'))
  pred_mask = torch.tensor(pred_mask, device = torch.device('cuda'))

  return image, gt_mask, pred_mask #gt_mask could be all zero when the ground truth is not given.



### Visualization and Submission

In [ ]:

'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''

train_set = get_detection_data('train')
for i in np.random.randint(0, 190, 3):
  img, gt_mask, pred_mask = get_prediction_mask(train_set[i])
  
  pred_mask = pred_mask.cpu().numpy()
  gt_mask = gt_mask.cpu().numpy()

  value_pred = 255./(pred_mask.max())
  pred_mask = pred_mask*value_pred
 
  value_gt = 255./(gt_mask.max())
  gt_mask = gt_mask*value_gt

  cv2_imshow(cv2.resize(img, (img.shape[1], img.shape[0]), interpolation = cv2.INTER_AREA))
  cv2_imshow(cv2.resize(gt_mask, (gt_mask.shape[1], gt_mask.shape[0]), interpolation = cv2.INTER_AREA))
  cv2_imshow(cv2.resize(pred_mask, (pred_mask.shape[1], pred_mask.shape[0]), interpolation = cv2.INTER_AREA))


In [ ]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
my_data_list = DatasetCatalog.get("plane_{}".format('train'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

'''
# Writing the predictions of the test set
'''

my_data_list = DatasetCatalog.get("plane_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample)
  pred_mask = torch.tensor(pred_mask, device = torch.device('cuda'))

  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()


## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [ ]:
DatasetCatalog.remove("plane_train")
DatasetCatalog.remove("plane_test")

In [ ]:
for d in ["train", "test"]:
  DatasetCatalog.register("plane_" + d, lambda d = d: get_detection_data(d))
  MetadataCatalog.get("plane_" + d).set(thing_classes=["plane"])
plane_metadata = MetadataCatalog.get("plane_train")

### Network

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("plane_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 600
cfg.SOLVER.STEPS = []        
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)

### Training

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume = False)
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

### Evaluation and Visualization

In [ ]:
dataset_dicts = get_detection_data("test")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=plane_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
evaluator = COCOEvaluator("plane_train", output_dir=cfg.OUTPUT_DIR)
train_loader = build_detection_test_loader(cfg, "plane_train")
print(inference_on_dataset(predictor.model, train_loader, evaluator))